In [81]:
import pandas as pd

Input the two csv files

In [82]:
flowCardHolders = pd.read_csv("Flow Card Holders.csv")
nonFlowCardHolders = pd.read_csv("Non-Flow Card Holders.csv")

Union the files together

In [83]:
df = pd.concat([flowCardHolders, nonFlowCardHolders], ignore_index=True)

Convert the Date field to a Quarter Number instead

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df['Quarter'] = df['Date'].dt.quarter
df.head()

Aggregate the data in the following ways:

1. Median price per Quarter, Flow Card? and Class
2. Minimum price per Quarter, Flow Card? and Class
3. Maximum price per Quarter, Flow Card? and Class

In [104]:
median_df = df.groupby(['Quarter','Class', 'Flow Card?'])['Price'].median().reset_index()
minimum_df = df.groupby(['Quarter','Class', 'Flow Card?'])['Price'].min().reset_index()
maximum_df = df.groupby(['Quarter','Class', 'Flow Card?'])['Price'].max().reset_index()

In [105]:
minimum_df.head(50)

,Quarter,Class,Flow Card?,Price
0,1,Business Class,No,241.2
1,1,Business Class,Yes,249.6
2,1,Economy,No,1030.0
3,1,Economy,Yes,1020.0
4,1,First Class,No,204.0
5,1,First Class,Yes,201.0
6,1,Premium Economy,No,515.0
7,1,Premium Economy,Yes,502.5
8,2,Business Class,No,240.0
9,2,Business Class,Yes,240.0


Now pivot the data to have a column per class for each quarter and whether the passenger had a flow card or not

In [108]:
pivot_min = minimum_df.pivot_table(index = ['Flow Card?','Quarter'],
                                columns= 'Class',
                                values = 'Price',
                                aggfunc= 'min',
                                fill_value=0).reset_index()

pivot_max = maximum_df.pivot_table(index = ['Flow Card?','Quarter'],
                                columns= 'Class',
                                values = 'Price',
                                aggfunc='max',
                                fill_value=0).reset_index()

pivot_median = median_df.pivot_table(index = ['Flow Card?','Quarter'],
                                columns= 'Class',
                                values = 'Price',
                                aggfunc='median',
                                fill_value=0).reset_index()



Class,Flow Card?,Quarter,Business Class,Economy,First Class,Premium Economy
0,No,1,241.2,1030.0,204.0,515.0
1,No,2,240.0,1000.0,202.0,507.5
2,No,3,240.0,1000.0,201.0,517.5
3,No,4,240.0,1015.0,200.0,510.0
4,Yes,1,249.6,1020.0,201.0,502.5
5,Yes,2,240.0,1020.0,200.0,500.0
6,Yes,3,241.2,1005.0,206.0,502.5
7,Yes,4,249.6,1030.0,205.0,505.0


Union these flows back together

In [110]:
pivot_max['Aggregation'] = 'max'
pivot_median['Aggregation'] = 'median'
pivot_min['Aggregation'] = 'min'


out = pd.concat([pivot_min, pivot_median, pivot_max], ignore_index=True)

In [111]:
out.head(30)

Class,Flow Card?,Quarter,Business Class,Economy,First Class,Premium Economy,Aggregation
0,No,1,241.2,1030.0,204.0,515.00,min
1,No,2,240.0,1000.0,202.0,507.50,min
2,No,3,240.0,1000.0,201.0,517.50,min
3,No,4,240.0,1015.0,200.0,510.00,min
4,Yes,1,249.6,1020.0,201.0,502.50,min
5,Yes,2,240.0,1020.0,200.0,500.00,min
6,Yes,3,241.2,1005.0,206.0,502.50,min
7,Yes,4,249.6,1030.0,205.0,505.00,min
8,No,1,574.8,2340.0,438.0,1075.00,median
9,No,2,553.8,2325.0,445.0,1205.00,median


In [113]:
out.rename(columns={'Economy' : 'First', 'First Class' : 'Economy', 'Business Class' : 'Premium', 'Premium Economy':'Business'}, inplace=True)

In [114]:
out.head(30)

Class,Flow Card?,Quarter,Premium,First,Economy,Business,Aggregation
0,No,1,241.2,1030.0,204.0,515.00,min
1,No,2,240.0,1000.0,202.0,507.50,min
2,No,3,240.0,1000.0,201.0,517.50,min
3,No,4,240.0,1015.0,200.0,510.00,min
4,Yes,1,249.6,1020.0,201.0,502.50,min
5,Yes,2,240.0,1020.0,200.0,500.00,min
6,Yes,3,241.2,1005.0,206.0,502.50,min
7,Yes,4,249.6,1030.0,205.0,505.00,min
8,No,1,574.8,2340.0,438.0,1075.00,median
9,No,2,553.8,2325.0,445.0,1205.00,median


In [115]:
out.to_csv("AggregatedFile.csv",index=False)